## PyPSA Introduction Notebook

### 🎯 Objective

This notebook provides a deeper dive into the PyPSA toolbox using a network generated from a **PyPSA-Earth** workflow.

We work with two versions of the same network:
- **Pre-network**: before solving the optimization problem  
- **Post-network**: after solving, containing results

Since solving these models can be computationally intensive, both versions are provided for convenience.

In this session, we will explore the more detailed network model and introduce the **PyPSA statistics library** to analyze system-level results.



### ✍️ Authors
<div style="display: flex; align-items: center; justify-content: space-between; flex-wrap: nowrap; gap: 2rem;">

  <div style="flex: 1; min-width: 250px;">
      <a href="https://www.linkedin.com/in/virio-andreyana" target="_blank">Virio Andreyana</a><br>
      <a href="https://www.linkedin.com/in/andreas-denyer" target="_blank">Andreas Denyer</a><br>
      <a href="https://www.linkedin.com/in/priyeshgosai" target="_blank">Priyesh Gosai</a>
    </p>
  </div>

  <div style="flex-shrink: 0;">
    <a href="https://openenergytransition.org/index.html" target="_blank">
      <img src="https://openenergytransition.org/assets/img/oet-logo-red-n-subtitle.png" height="60" alt="Open Energy Transition logo">
    </a>
  </div>

</div>


### ⚙️ Setup Environment

In [ ]:
# @title Install the required packages
# @markdown Run this cell to install the necessary Python packages for the project.
import subprocess, sys, importlib

try:
    from importlib import metadata  # Python 3.8+
except ImportError:
    import importlib_metadata as metadata  # Backport for older versions

packages = [
    "pypsa",        # power-system modelling & optimization toolbox
    "pypsa[excel]", # pypsa with Excel I/O support
    "folium",       # interactive leaflet-based maps in Python
    "mapclassify"   # spatial data classification for choropleth maps
]

# Install packages
subprocess.run(
    [sys.executable, "-m", "pip", "install", "-q", *packages],
    check=True
)

# Check and report installed versions
base_names = [pkg.split("[")[0] for pkg in packages]
missing = [p for p in base_names if importlib.util.find_spec(p) is None]

if not missing:
    print("✅ All packages installed successfully.\n")
    for pkg in base_names:
        try:
            version = metadata.version(pkg)
            print(f"📦 {pkg} version {version}")
        except metadata.PackageNotFoundError:
            print(f"⚠️ {pkg} is not found in metadata.")
else:
    print(f"❌ Missing: {', '.join(missing)}")



### 🗃️ Data in the PyPSA Object

In [ ]:
# @title Download prepared networks
# @markdown We have prepared a case study that can be explored in this notebook. The case is based on the electircity network from PyPSA. To allow users to interact with the data we have prepared two networks. One before the optimization and another after the optimization. Running this optimization is a computationally expensive task.
# @markdown * `pre-network.nc`: The network is prepared before solving it.
# @markdown * `post-network.nc`: The solved network

from urllib.request import urlretrieve

urls = {
    "pre-network.nc": "https://drive.usercontent.google.com/download?id=17b7YZGXKczY2K5sRPUDJkD5AVwgbOgAh&export=download",
    "post-network.nc": "https://drive.usercontent.google.com/download?id=1qIN0tlZBACPtKCBxHUpBAecBqYsy-sTV&export=download&confirm=t&uuid=cf9cb5cf-de33-4ef4-9f49-5f01f2d571b1",
    }
for name, url in urls.items():
    print(f"Retrieving {name} from Google Drive")
    urlretrieve(url, name)
print("Done")

Import the `pre-network` to an object called `pre_network`

In [ ]:
import pypsa
import pandas as pd
pre_network = pypsa.Network("pre-network.nc")

Inspect the data in the network.

_All data in a PyPSA model is stored as pandas DataFrames, allowing you to apply any standard pandas operations directly to PyPSA network components._

In [ ]:
pre_network.buses

In [ ]:
pre_network.generators

In [ ]:
pre_network.generators_t.p_max_pu.head()

In [ ]:
pre_network.generators_t.p_max_pu.plot()

In [ ]:
pre_network.links

In [ ]:
pre_network.lines

In [ ]:
pre_network.loads

If you wanted to solve the network we could run:
```python
pre_network.optimize(solver_name = 'gurobi') # Assuming that you have a licence.
```

The solved network data is imported from the file `post-network.nc`

In [ ]:
import pypsa
import pandas as pd
post_network = pypsa.Network("post-network.nc")

#### 🧮 Results

---

📊 `pypsa.statistics`

The `pypsa.statistics` module provides a set of high-level functions for analyzing and summarizing the results of PyPSA models. It enables users to compute key metrics such as:

- **Generation mix** by technology  
- **Installed capacity** summaries  
- **Line loading** and **congestion metrics**  
- **Costs and revenues** by component  
- **CO₂ emissions** and carbon intensity  
- **Regional energy balances** and **power flows**

These functions are especially useful for post-processing model results and producing plots or reports for scenario analysis, capacity expansion studies, and policy evaluation.

The module works directly with the PyPSA `Network` object and leverages pandas operations internally, so results are returned as DataFrames that can be easily visualized or exported.




In [ ]:
s = post_network.statistics

You can easily have an comprehensive overview of the system level results.

In [ ]:
s().head()

Let's have a look to optimal renewable capacities.

In [ ]:
(
    s.optimal_capacity(
        bus_carrier=["AC", "low voltage"],
        comps="Generator",
    ).div(
        1e3
    )  # GW
)

You can get it as fancy as you want!

In [ ]:
(
    s.optimal_capacity(
        bus_carrier=["AC", "low voltage"],
        groupby=["location", "carrier"],
        comps="Generator",
    )
    .div(1e3)  # GW
    .to_frame(name="p_nom_opt")
    .pivot_table(index="location", columns="carrier", values="p_nom_opt")
    .fillna(0)
    .assign(Total=lambda df: df.sum(axis=1))
    .sort_values(by="Total", ascending=False)
    .round(2)
).head()

We can also easily look into the energy balance for a specific carrier by Node.

So, let's investigate the Hydrogen balance at the Z1 and Z2 nodes of Germany (DE):

In [ ]:
df = (
    s.energy_balance(groupby=["bus_carrier", "country", "bus", "carrier", "name"])
    .div(1e6)  # TWh
    .to_frame(name="Balance [TWh]")
    .query(
        "(bus_carrier.str.contains('Hydrogen')) "
        "and (country == 'DE') "
        " and (abs(`Balance [TWh]`) > 1e-2)"
    )
    .round(2)
)
df

In [ ]:
# verify energy balance
df.groupby(by="bus").sum()

In [ ]:
exports = df.query("name.str.contains('DE ->')")
export_twh = exports["Balance [TWh]"].sum().round(2)
print(f"DE exports {export_twh} TWh of H2.")

imports = df.query(
    "(name.str.contains('-> DE')) and not (name.str.contains('Z1')) and not (name.str.contains('Z2'))"
)
import_twh = imports["Balance [TWh]"].sum().round(2)
print(f"DE imports {import_twh} TWh of H2.")

balance_twh = import_twh + export_twh
print(
    f"DE is a net {'importer' if balance_twh > 0 else 'exporter'} ({balance_twh.round(2)} TWh)."
)

... or look at renewable curtailment in the system:

In [ ]:
(
    s.curtailment(
        bus_carrier=["AC", "low voltage"],
        groupby=["location", "carrier"],
    )
    .div(1e6)  # TWh
    .to_frame(name="p_nom_opt")
    .pivot_table(index="location", columns="carrier", values="p_nom_opt")
    .fillna(0)
    .assign(Total=lambda df: df.sum(axis=1))
    .sort_values(by="Total", ascending=False)
    .round(2)
).head()

---
📈 `pypsa.plot`

PyPSA also includes a built-in `pypsa.plot` module that provides a small set of standard plotting functions. These are useful for quickly visualizing key aspects of your network, such as:

- Network topology (buses, lines, generators)
- Line loading and power flows
- Generation dispatch over time
- Installed capacities and time-series data

These plots are useful for exploratory analysis and debugging and can be customized using `matplotlib`.

For more advanced or interactive visualizations, you may consider combining PyPSA outputs with external libraries such as `plotly`, `holoviews`, or `hvplot`.


In [ ]:
# let's fill missing colors first
pd.options.plotting.backend = "matplotlib"
post_network.carriers.loc["none", "color"] ="#000000"
post_network.carriers.loc["", "color"] = "#000000"

Let's now plot the optimal renewable capacities that we investigated before.

In [ ]:
s.optimal_capacity.plot.bar(
    bus_carrier="AC",
    query="value>1e3",
    height=6,
);

You can also have details for specific countries.

In [ ]:
s.optimal_capacity.plot.bar(
    bus_carrier="AC",
    query="value>1e3 and country in ['DE', 'FR']",
    height=6,
    facet_col="country",
);

You can have a closer look to the wind production

In [ ]:
s.energy_balance.plot.line(
    facet_row="bus_carrier",
    y="value",
    x="snapshot",
    carrier="wind",
    nice_names=False,
    color="carrier",
    aspect=5.0,
);

... or to the dispatch for specific countries.

In [ ]:
s.energy_balance.plot.area(
    bus_carrier=["AC"],
    y="value",
    x="snapshot",
    color="carrier",
    stacked=True,
    facet_row="country",
    query="country in ['DE', 'FR'] and snapshot < '2013-03'",
    aspect=5,
);

You can also explore H2 results.

In [ ]:
s.energy_balance.plot.bar(
    bus_carrier=["H2"],
    y="carrier",
    x="value",
    color="carrier",
    facet_col="country",
    height=4,
    aspect=1,
    query="country in ['DE', 'FR']",
);

You can also explore the correlation between renewable production and hydrogen.

In [ ]:
s.energy_balance.plot.area(
    bus_carrier=["AC", "H2"],
    y="value",
    x="snapshot",
    color="carrier",
    stacked=True,
    facet_row="bus_carrier",
    sharex=False,
    sharey=False,
    query="snapshot < '2013-03'",
    aspect=5,
);

---